<h1 style='text-align: center;'>Survivor:</h1>
<h2 style='text-align: center;'>Exploring Contestant Trends Through Data</h2>

##### This projects aims to look at the long-running CBS show Survivor over the years, and determine what demographics of contestants are more likely to win.
##### We will also be looking to see if there are any trends where certain demographics have lower chances of winning.


In [3]:
import pandas as pd
import matplotlib as plt